In [13]:
#Biomechanics coursework 1
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib qt

from IPython.core.debugger import set_trace

In [2]:
Force_1_data = pd.read_excel('Force_plate_data_2019.xlsx', sheet_name='Forceplate_1')
Force_2_data = pd.read_excel('Force_plate_data_2019.xlsx', sheet_name='Forceplate_2')
Force_1_data.head()

,Sample,Time,Force_X,Force_Y,Force_Z,Moment_X,Moment_Y,Moment_Z,COP_X,COP_Y,COP_Z
0,2336,2.336,-0.375547,-0.402564,-7.189139,0.991663,0.201020,0.202878,0.0,0.0,0
1,2337,2.337,-0.415633,-0.323380,-5.313049,0.991665,0.200213,0.200279,0.0,0.0,0
2,2338,2.338,-0.416249,-0.283952,-4.062634,1.647962,0.068014,0.168852,0.0,0.0,0
3,2339,2.339,-0.415325,-0.441910,-5.628289,0.662005,0.397203,0.203556,0.0,0.0,0
4,2340,2.340,-0.536507,-0.402482,-5.623490,0.882337,0.003425,0.186051,0.0,0.0,0


In [36]:
def plot_force_plate(all_data):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    ax.quiver(all_data['COP_X'], all_data['COP_Y'], all_data['COP_Z'],
              all_data['Force_X'], all_data['Force_Y'], all_data['Force_Z'],
              arrow_length_ratio=0.01, length=3*10e-3) #default arrow length is way too long

    ax.set_title('Gait analysis force plate data', fontsize='large', fontweight='bold')
    ax.set_xlabel('x (mm)')
    ax.set_ylabel('y (mm)')
    ax.set_zlabel('z (mm)')
    ax.set_zlim(-0.005, 30)
    plt.show()

In [37]:
plot_force_plate(Force_1_data)
#plot_force_plate(Force_2_data)

In [5]:
Forceplate_1_info = pd.read_excel('Force_plate_data_2019.xlsx', sheet_name='Forceplate_1_info')
Forceplate_2_info = pd.read_excel('Force_plate_data_2019.xlsx', sheet_name='Forceplate_2_info')
Forceplate_1_info.T

,0
NO_OF_SAMPLES,1724.000000
FREQUENCY,1000.000000
FIRST_SAMPLE,2333.000000
FORCE_PLATE_CORNER_POSX_POSY_X,881.198975
FORCE_PLATE_CORNER_POSX_POSY_Y,581.159973
FORCE_PLATE_CORNER_POSX_POSY_Z,-4.928020
FORCE_PLATE_CORNER_NEGX_POSY_X,880.101990
FORCE_PLATE_CORNER_NEGX_POSY_Y,-2.366100
FORCE_PLATE_CORNER_NEGX_POSY_Z,-5.842590
FORCE_PLATE_CORNER_NEGX_NEGY_X,-0.813263


In [6]:
#commpute center of force plate
plate_center = [[], [], []]
num_gen = (i for i in range(3))
Forceplate_list = [Forceplate_1_info, Forceplate_2_info]

for i in Forceplate_list:
    num = next(num_gen)
    for j in ['_X','_Y','_Z']:
        this_data = i.filter(like=j).T
        plate_center[num].append(np.mean(this_data))

plate_center = [pd.concat(plate_center[i], axis=0) for i in [0,1]]

In [7]:
Force_list = [Force_1_data, Force_2_data]
adjusted = [[], []]

for i in range(2):
    this_data = Force_list[i]
    adjusted[i].append(this_data['COP_X'] + plate_center[i].iloc[1]) #add lab y to force plate x
    adjusted[i].append(this_data['COP_Y'] + plate_center[i].iloc[0]) #add lab x to force plate y
    adjusted[i].append(this_data['COP_Z'] + plate_center[i].iloc[2])

In [49]:
#method 1: add center coordinate of force plate to COP values

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

for i in range(2):
    ax.quiver(adjusted[i][0], adjusted[i][1], adjusted[i][2],
              Force_list[i]['Force_X'], Force_list[i]['Force_Y'], Force_list[i]['Force_Z'],
              arrow_length_ratio=0.01, length=4*10e-3) #default arrow length is way too long

ax.set_title('Gait analysis force plate data', fontsize='large', fontweight='bold')
ax.set_xlabel('x (mm)')
ax.set_ylabel('y (mm)')
ax.set_zlabel('z (mm)')
ax.set_xlim(100, 800)
ax.set_ylim(100, 1800)
ax.set_zlim(-0.05, 30)
plt.show()

In [9]:
#method 2: transformation matrix

#initialize transformation matrix
# T = [np.zeros((4,4))] * 2 #why does this copy the two things ???!!!!!!
T = [np.zeros((4,4)), np.zeros((4,4))]

for i in range(2):
    #set_trace()
    this_forceplate = Forceplate_list[i]
    
    #vector 1
    v1 = np.array([this_forceplate['FORCE_PLATE_CORNER_NEGX_NEGY_X'] - this_forceplate['FORCE_PLATE_CORNER_POSX_NEGY_X'],
                   this_forceplate['FORCE_PLATE_CORNER_NEGX_NEGY_Y'] - this_forceplate['FORCE_PLATE_CORNER_POSX_NEGY_Y'],
                   this_forceplate['FORCE_PLATE_CORNER_NEGX_NEGY_Z'] - this_forceplate['FORCE_PLATE_CORNER_POSX_NEGY_Z']]).reshape(1,-1)
    T[i][:3, 0] = v1 / np.linalg.norm(v1)
    
    #vector 3
    v3 = np.cross(T[i][:3, 0], np.array([this_forceplate['FORCE_PLATE_CORNER_POSX_POSY_X'] - this_forceplate['FORCE_PLATE_CORNER_POSX_NEGY_X'],
                                         this_forceplate['FORCE_PLATE_CORNER_POSX_POSY_Y'] - this_forceplate['FORCE_PLATE_CORNER_POSX_NEGY_Y'],
                                         this_forceplate['FORCE_PLATE_CORNER_POSX_POSY_Z'] - this_forceplate['FORCE_PLATE_CORNER_POSX_NEGY_Z']]).reshape(1,-1))
    T[i][:3, 2] = v3 / np.linalg.norm(v3)
    
    #vector 2
    T[i][:3, 1] = np.cross(T[i][:3, 2], T[i][:3, 0])
    
    #vector 4 (position vector)
    T[i][:3, 3] = np.array([this_forceplate['FORCE_PLATE_CORNER_NEGX_NEGY_X'],
                            this_forceplate['FORCE_PLATE_CORNER_NEGX_NEGY_Y'],
                            this_forceplate['FORCE_PLATE_CORNER_NEGX_NEGY_Z']]).reshape(1,-1)
    
    #row 4                  
    T[i][3, :] = np.array([0, 0, 0, 1])

T

[array([[ 6.85728886e-03,  9.99975503e-01, -1.40380042e-03,
         -8.13263000e-01],
        [-9.99976484e-01,  6.85741112e-03,  8.22986087e-05,
         -6.94627000e+00],
        [ 9.19230293e-05,  1.40320306e-03,  9.99999011e-01,
         -6.11779000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          1.00000000e+00]]),
 array([[ 3.01010268e-03,  9.99995438e-01,  2.52933914e-04,
          9.02068970e+02],
        [-9.99992803e-01,  3.01067864e-03, -2.30845806e-03,
         -2.48950000e+00],
        [-2.30920903e-03, -2.45983398e-04,  9.99997304e-01,
         -6.30485000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          1.00000000e+00]])]

In [11]:
#apply trnasformation

force_transformed_list = []
COP_transformed_list = []

for i in range(2):
    this_data = Force_list[i]
    force = np.array(this_data[['Force_X', 'Force_Y', 'Force_Z']])
    force = np.concatenate((force, np.array([1]*len(force)).reshape(-1,1)), axis=1)
    
    COP = np.array(this_data[['COP_X', 'COP_Y', 'COP_Z']])
    COP = np.concatenate([COP, np.array([1]*len(COP)).reshape(-1,1)], axis=1)
    
    force_transformed = [T[i] @ force[j,:] for j in range(len(force))]
    COP_transformed = [T[i] @ COP[j,:] for j in range(len(COP))]
    
    force_transformed = np.concatenate(force_transformed, axis=0).reshape(-1, 4)    
    COP_transformed = np.concatenate(COP_transformed, axis=0).reshape(-1, 4)  
    
    force_transformed_list.append(force_transformed)
    COP_transformed_list.append(COP_transformed)

In [58]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

for i in range(2):
    ax.quiver(COP_transformed_list[i][:,0], COP_transformed_list[i][:,1], COP_transformed_list[i][:,2],
              force_transformed_list[i][:,0], force_transformed_list[i][:,1], force_transformed_list[i][:,2],
              arrow_length_ratio=0.01, length=6*10e-3) #default arrow length is way too long

ax.set_title('Gait analysis force plate data', fontsize='large', fontweight='bold')
ax.set_xlabel('x (mm)')
ax.set_ylabel('y (mm)')
ax.set_zlabel('z (mm)')
ax.set_xlim(-200, 1500)
ax.set_ylim(-200, 500)
ax.set_zlim(-10, 50)
plt.show()